# Import Libraries

In [1]:
# Standard libraries
import os
import shutil

# Azure ML libraries
import azureml.core
from azureml.core import Workspace, Model
from azureml.widgets import RunDetails
from azureml.core.resource_configuration import ResourceConfiguration

# Azure ML Setup

In [2]:
# Check core SDK version number
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK.")
print("")

# Load workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

You are currently using version 1.0.83 of the Azure ML SDK.

Workspace name: sbirkamlws
Azure region: northcentralus
Subscription id: bf088f59-f015-4332-bd36-54b988be7c90
Resource group: sbirkamlrg


# Create Files for Experiment

In [3]:
# Create a folder for the experiment files
training_folder = 'model_training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/customer_churn.csv', os.path.join(training_folder, "customer_churn.csv"))

'model_training/customer_churn.csv'

In [4]:
# Create the training script in the experiment folder

In [5]:
%%writefile $training_folder/customer_churn_training.py

### Setup
## Import libraries
# Standard libraries
import pandas as pd
import numpy as np
import joblib
import argparse

# Azure ML libraries
from azureml.core import Run

# Data preprocessing libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Model training libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import scorer, f1_score, precision_score, recall_score, plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import statsmodels.api as sm

# Visualization libraries
import plotly.graph_objs as go
import plotly.subplots as sp
import plotly.offline as py
import plotly.io as pio
import plotly
import matplotlib.pyplot as plt

# Create outputs folder
os.makedirs('outputs', exist_ok=True)

# configure plotly.py to run orca using Xvfb
pio.orca.config.use_xvfb = True
pio.orca.config.save()

# Get the experiment run context
run = Run.get_context()

# Load the dataset
print("Loading Data...")
telcom = pd.read_csv('customer_churn.csv')


### Data preprocessing
# Replace spaces with null values in total charges column
telcom['TotalCharges'] = telcom["TotalCharges"].replace(" ",np.nan)

# Drop null values from total charges column which contains .15% missing data 
telcom = telcom[telcom["TotalCharges"].notnull()]
telcom = telcom.reset_index()[telcom.columns]

# Convert total charges to float type
telcom["TotalCharges"] = telcom["TotalCharges"].astype(float)

# Replace 'No internet service' to No for the following columns
replace_cols = ["OnlineSecurity", "OnlineBackup", "DeviceProtection",
                "TechSupport","StreamingTV", "StreamingMovies"]
for i in replace_cols : 
    telcom[i]  = telcom[i].replace({"No internet service" : "No"})
    
# Replace values for Senior Citizen column
telcom["SeniorCitizen"] = telcom["SeniorCitizen"].replace({1:"Yes",0:"No"})

# Transform Tenure to categorical column
def tenure_lab(telcom) :    
    if telcom["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (telcom["tenure"] > 12) & (telcom["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (telcom["tenure"] > 24) & (telcom["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (telcom["tenure"] > 48) & (telcom["tenure"] <= 60) :
        return "Tenure_48-60"
    elif telcom["tenure"] > 60 :
        return "Tenure_gt_60"
    
telcom["tenure_group"] = telcom.apply(lambda telcom:tenure_lab(telcom), axis = 1)

# ID column
Id_col     = ['customerID']

# Target column
target_col = ["Churn"]

# Categorical feature columns
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist() # get columns with less than 6 unique values
cat_cols   = [x for x in cat_cols if x not in target_col] # exclude target column (which is also categorical)

# Numerical feature columns
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

# Binary columns with two values
bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()

# Columns with more than two values
multi_cols = [i for i in cat_cols if i not in bin_cols]

# Label encoding of binary columns
le = LabelEncoder()
for i in bin_cols :
    telcom[i] = le.fit_transform(telcom[i])
    
# Build dummy columns for multi value columns
telcom = pd.get_dummies(data = telcom,columns = multi_cols)

# Scaling of numerical columns
std = StandardScaler()
scaled = std.fit_transform(telcom[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

# Dropping original values and merging scaled values for numerical columns
telcom = telcom.drop(columns = num_cols,axis = 1)
telcom = telcom.merge(scaled,left_index=True,right_index=True,how = "left")


### Model Training
# Split data into train and test
train,test = train_test_split(telcom, test_size = .25 ,random_state = 42)
    
# Separate dependent and independent variables and exclude ID column
cols    = [i for i in telcom.columns if i not in Id_col + target_col]
train_X = train[cols]
train_Y = train[target_col]
test_X  = test[cols]
test_Y  = test[target_col]

# Write a function for generic model training    
def telecom_churn_prediction(algorithm,training_x,testing_x,
                             training_y,testing_y,cols,cf):
    '''
    Function signature
    algorithm     - algorithm used 
    training_x    - predictor variables dataframe (training)
    testing_x     - predictor variables dataframe (testing)
    training_y    - target variable (training)
    training_y    - target variable (testing)
    cols          - features
    cf - ["coefficients","features"] (cooefficients for logistic regression, features for tree based models)
    '''
    
    # Fit the model
    algorithm.fit(training_x,training_y)
    predictions   = algorithm.predict(testing_x)
    probabilities = algorithm.predict_proba(testing_x)
    
    # Store coefficients
    if   cf == "coefficients" :
        coefficients  = pd.DataFrame(algorithm.coef_.ravel())
    elif cf == "features" :
        coefficients  = pd.DataFrame(algorithm.feature_importances_)
        
    column_df     = pd.DataFrame(cols)
    coef_sumry    = (pd.merge(coefficients,column_df,left_index= True,
                              right_index= True, how = "left"))
    coef_sumry.columns = ["coefficients","features"]
    coef_sumry    = coef_sumry.sort_values(by = "coefficients",ascending = False)
    
    # Evaluate the model
    class_report = classification_report(testing_y,predictions)
    class_metrics = precision_recall_fscore_support(testing_y, predictions, beta=1.0, average="binary", pos_label=0)
    accuracy = accuracy_score(testing_y,predictions)
    conf_matrix = confusion_matrix(testing_y,predictions)
    model_roc_auc = roc_auc_score(testing_y,predictions)
    
    precision = class_metrics[0]
    recall = class_metrics[1]
    fscore = class_metrics[2]
    
    # Log and print model evaluation information
    print (algorithm)
    
    print ("\n Classification report : \n", class_report)
    
    print ("Accuracy Score : ", accuracy)
    run.log("Accuracy", np.float(accuracy))    
    
    print ("Area Under Curve : ",model_roc_auc,"\n")
    run.log("AUC", np.float(model_roc_auc))
    
    print ("Precision : ",precision,"\n")
    run.log("Precision", np.float(precision))
    
    print ("Recall : ",recall,"\n")
    run.log("Recall", np.float(recall))
    
    print ("F Score : ",fscore,"\n")
    run.log("F Score", np.float(fscore))
    
    fpr,tpr,thresholds = roc_curve(testing_y,probabilities[:,1])
    
    # Plot confusion matrix
    class_names = ["Churn", "Non Churn"]
    
    trace1 = plot_confusion_matrix(algorithm, testing_x, testing_y,
                                   display_labels=class_names,
                                   cmap="inferno",
                                   normalize="true")
    
    trace1.ax_.set_title("Normalized Confusion Matrix")
    trace1.ax_.grid(False)
    run.log_image("Normalized Confusion Matrix", plot=plt)
    
    plt.clf()
    
    # Plot roc curve
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % model_roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    run.log_image("Receiver Operating Characteristic", plot=plt)
    
    plt.clf()
    
    # Plot feature importance
    x = coef_sumry["features"]
    y = coef_sumry["coefficients"]

    mask1 = y >= 0
    mask2 = y < 0

    objects = coef_sumry["features"]
    x_pos = np.arange(len(coef_sumry["features"]))

    plt.bar(x[mask1], y[mask1], color = 'green')
    plt.bar(x[mask2], y[mask2], color = 'red')
    plt.xticks(x_pos, x, rotation='vertical')
    plt.yticks(rotation='horizontal')
    plt.ylabel('Feature Coefficient')
    plt.title('Feature Importance')
    plt.tight_layout()
    run.log_image("Feature Importance", plot=plt)
    
    plt.clf()
    
    '''
    # Plot confusion matrix
    trace1 = go.Heatmap(z = conf_matrix ,
                        x = ["Not churn","Churn"],
                        y = ["Not churn","Churn"],
                        showscale  = False,colorscale = "Picnic",
                        name = "matrix")
    
    # Plot roc curve
    trace2 = go.Scatter(x = fpr,y = tpr,
                        name = "Roc : " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'),width = 2))
    trace3 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(205, 12, 24)'),width = 2,
                        dash = 'dot'))
    
    # Plot coefficients
    trace4 = go.Bar(x = coef_sumry["features"],y = coef_sumry["coefficients"],
                    name = "coefficients",
                    marker = dict(color = coef_sumry["coefficients"],
                                  colorscale = "Picnic",
                                  line = dict(width = .6,color = "black")))
    
    # Create subplots
    fig = sp.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                            subplot_titles=('Confusion Matrix',
                                            'Receiver operating characteristic',
                                            'Feature Importances'))
    
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,2)
    fig.append_trace(trace3,1,2)
    fig.append_trace(trace4,2,1)
    
    # Plot layout
    fig['layout'].update(showlegend=False, title="Model performance" ,
                         autosize = False,height = 900,width = 800,
                         plot_bgcolor = 'rgba(240,240,240, 0.95)',
                         paper_bgcolor = 'rgba(240,240,240, 0.95)',
                         margin = dict(b = 195))
    fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
    fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
    fig["layout"]["xaxis3"].update(dict(showgrid = True,tickfont = dict(size = 10),
                                        tickangle = 90))
    
    image_path = "outputs/log_reg_graphs.png"
    fig.write_image(image_path)
    # upload the file explicitly into artifacts 
    run.upload_file(name = image_path, path_or_stream = image_path)
    '''
    
    return algorithm
        
# Build logistic regression model

# Expose regularization hyperparameter as script argument
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

run.log('Regularization Hyperparameter Lambda', np.float(reg)) # this is retrieved as script argument

logit = LogisticRegression(C=1/reg, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

logit_fitted = telecom_churn_prediction(logit,train_X,test_X,train_Y,test_Y,
                         cols,"coefficients")

# Save the trained model in the outputs folder
joblib.dump(value=logit_fitted, filename='outputs/customer_churn_log_reg_model.pkl')

run.complete()

Overwriting model_training/customer_churn_training.py


# Use an Estimator to Run the Script as an Experiment

In [6]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment

# Create an estimator
estimator = Estimator(source_directory=training_folder,
                      entry_script='customer_churn_training.py',
                      script_params = {'--reg_rate': 0.01},
                      compute_target='local',
                      conda_packages=['scikit-learn', 'statsmodels', 'plotly', 'psutil'],
                      pip_packages=['matplotlib']
                      )

# Create an experiment
experiment_name = 'customer-churn-log-reg-experiment'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

RunId: customer-churn-log-reg-experiment_1580814694_86381982
Web View: https://ml.azure.com/experiments/customer-churn-log-reg-experiment/runs/customer-churn-log-reg-experiment_1580814694_86381982?wsid=/subscriptions/bf088f59-f015-4332-bd36-54b988be7c90/resourcegroups/sbirkamlrg/workspaces/sbirkamlws

Streaming azureml-logs/70_driver_log.txt

Starting the daemon thread to refresh tokens in background for process with pid = 10
Entering Run History Context Manager.
/azureml-envs/azureml_b801e5ddb9f8efe313713a85e8a72474/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
Loading Data...
/azureml-envs/azureml_b801e5ddb9f8efe313713a85e8a72474/lib/python3.6/

{'runId': 'customer-churn-log-reg-experiment_1580814694_86381982',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-02-04T11:11:37.054446Z',
 'endTimeUtc': '2020-02-04T11:11:47.248712Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '7284c6c9-990f-405d-ad22-a1490fa705eb'},
 'inputDatasets': [],
 'runDefinition': {'script': 'customer_churn_training.py',
  'useAbsolutePath': False,
  'arguments': ['--reg_rate', '0.01'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'environment': {'name': 'Experiment customer-churn-log-reg-experiment Environment',
   'version': 'Autosave_2020-02-04T11:06:30Z_8ccdb337',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'condaDependencies': {'channels': ['conda-forge'],
     'dependencies': ['python=3.6.

In [7]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

# Register the Trained Model

In [8]:
# Register the model
run.register_model(model_path='outputs/customer_churn_log_reg_model.pkl', model_name='customer_churn_log_reg_model',
                   tags={'Training Context':'Estimator', 'Algorithm':'Logistic Regression'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy'],
                              'Precision': run.get_metrics()['Precision'], 'Recall': run.get_metrics()['Recall'],
                              'F Score': run.get_metrics()['F Score']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

customer_churn_log_reg_model version: 3
	 Training Context : Estimator
	 Algorithm : Logistic Regression
	 AUC : 0.6865149479341619
	 Accuracy : 0.78839590443686
	 Precision : 0.8290780141843972
	 Recall : 0.8992307692307693
	 F Score : 0.862730627306273


customer_churn_dt_model version: 2
	 Training Context : Notebook
	 Algorithm : Decision Tree


customer_churn_log_reg_model version: 2
	 Training Context : Estimator
	 Algorithm : Logistic Regression
	 AUC : 0.6865149479341619
	 Accuracy : 0.78839590443686
	 Precision : 0.8290780141843972
	 Recall : 0.8992307692307693
	 F Score : 0.862730627306273


AutoMLd0a94946323 version: 1


customer_churn_log_reg_model version: 1
	 Training Context : Estimator
	 Algorithm : Logistic Regression
	 AUC : 0.6894675848169298
	 Accuracy : 0.7906712172923777
	 Precision : 0.8304964539007093
	 Recall : 0.9007692307692308
	 F Score : 0.8642066420664208


customer_churn_dt_model version: 1
	 Training Context : Notebook
	 Algorithm : Decision Tree


